In [1]:
# %load lowlight_train.py
import torch
import torch.nn as nn
import torchvision
import torch.backends.cudnn as cudnn
import torch.optim
import os
import sys
import argparse
import time
import dataloader
import tifffile as tiff
import model
import Myloss_3D
import numpy as np
from torchvision import transforms

def get_img(path):    #读取tiff格式和其它格式图片，主要为3维16位图片
    if 'tif' in path:
        img = tiff.imread(path)
    else:
        img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    
    img = torch.tensor(img/65536)
    img = img[np.newaxis, np.newaxis, :, :, :]
    img = img[0:32, 0:32, 0:32]
    
#     img = img.permute(2, 0, 1)
#     img = img.to(device)
    
    return img


def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


def train(config):

    os.environ['CUDA_VISIBLE_DEVICES']='0'

#     DCE_net = model.enhance_net_nopool().cuda()

#     DCE_net.apply(weights_init)
#     if config.load_pretrain == True:
#         DCE_net.load_state_dict(torch.load(config.pretrain_dir))
#     train_dataset = dataloader.lowlight_loader(config.lowlight_images_path)

#     train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=config.train_batch_size, shuffle=True, num_workers=config.num_workers, pin_memory=True)


    # 初始化loss函数
    L_color = Myloss_3D.L_color()
    L_spa = Myloss_3D.L_spa()

    L_exp = Myloss_3D.L_exp(16,0.6)
    L_TV = Myloss_3D.L_TV()
    
    # 读取图片，其中enhance image = enhanced image

    img_path = 'data/883.tif'
    img = get_img(img_path)
    print('图像尺寸{}，图像类型{}'.format(img.shape, img.dtype))
    
    img_curve = np.random.rand(256)
    
    enhanced_image_1,enhanced_image,A  = img, img, None
    
    # loss部分
    A = torch.load('A')
    Loss_TV = 200*L_TV(A)

    
    loss_spa = torch.mean(L_spa(enhanced_image, img_lowlight))

    loss_col = 5*torch.mean(L_color(enhanced_image))

    loss_exp = 10*torch.mean(L_exp(enhanced_image))


    # best_loss
    loss =  Loss_TV + loss_spa + loss_col + loss_exp
    




if __name__ == "__main__":

    parser = argparse.ArgumentParser()

    # Input Parameters
    parser.add_argument('--lowlight_images_path', type=str, default="data/train_data/")
    parser.add_argument('--lr', type=float, default=0.0001)
    parser.add_argument('--weight_decay', type=float, default=0.0001)
    parser.add_argument('--grad_clip_norm', type=float, default=0.1)
    parser.add_argument('--num_epochs', type=int, default=200)
    parser.add_argument('--train_batch_size', type=int, default=8)
    parser.add_argument('--val_batch_size', type=int, default=4)
    parser.add_argument('--num_workers', type=int, default=4)
    parser.add_argument('--display_iter', type=int, default=10)
    parser.add_argument('--snapshot_iter', type=int, default=10)
    parser.add_argument('--snapshots_folder', type=str, default="snapshots/")
    parser.add_argument('--load_pretrain', type=bool, default= False)
    parser.add_argument('--pretrain_dir', type=str, default= "snapshots/Epoch99.pth")
    parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")

    config = parser.parse_args()

    if not os.path.exists(config.snapshots_folder):
        os.mkdir(config.snapshots_folder)


    train(config)








D:\anaconda\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
D:\anaconda\envs\pytorch\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: [WinError 126] 找不到指定的模块。
  warn(f"Failed to load image Python extension: {e}")


TypeError: not a sequence